# Usage

Because PRIMAP2 builds on xarray, all xarray functionality is available
right away.
Additional functionality is provided in the ``primap2`` package and
in the ``pr`` namespace on xarray objects.
In this section, we will present examples of PRIMAP2 usage.

## Importing

In [ ]:
import primap2  # injects the "pr" namespace into xarray

## Loading Datafiles

### Loading from netcdf files

The native storage format of PRIMAP2 are netcdf5 files, and datasets
can be written to and loaded from netcdf5 files using PRIMAP2 functions.
We will load the "opulent" Dataset from the data format section:

In [ ]:
ds = primap2.open_dataset("opulent_ds.nc")

ds

## Selecting Data

Of course, data can be selected using the
[xarray indexing methods](https://xarray.pydata.org/en/stable/indexing.html),
but PRIMAP2 also provides own versions of some of xarray's selection methods
which work using the dimension names without the category set.

### Getitem

The following selections both select the same:

In [ ]:
ds["area (ISO3)"]

In [ ]:
ds.pr["area"]

### The loc Indexer

Similarly, a version of the `loc` indexer is provided which works with the
bare dimension names:

In [ ]:
ds.pr.loc[{"time": slice("2002", "2005"), "animal": "cow"}]

It also works on DataArrays:

In [ ]:
da = ds["CO2"]

da.pr.loc[
    {
        "time": slice("2002", "2005"),
        "animal": "cow",
        "category": "0",
        "area": "COL",
    }
]

## Unit handling

PRIMAP2 uses the [openscm_units](https://openscm-units.readthedocs.io)
package based on the [Pint](https://pint.readthedocs.io/) library
for handling of units.

### CO2 equivalent units and mass units

Using global warming potential contexts, it is easy to convert mass units
into CO2 equivalents:

In [ ]:
from primap2 import ureg  # The unit registry

sf6_gwp = ds["SF6"].pr.convert_to_gwp(
    gwp_context="AR4GWP100", units="Gg CO2 / year"
)
# The information about the used GWP context is retained:
sf6_gwp.attrs

Because the GWP context used for conversion is stored, it is equally easy
to convert back to mass units:

In [ ]:
sf6 = sf6_gwp.pr.convert_to_mass()

The stored GWP context can also be used to convert another array using the
same context:

In [ ]:
ch4_gwp = ds["CH4"].pr.convert_to_gwp_like(sf6_gwp)

### Dropping units

Sometimes, it is necessary to drop the units, for example to use
arrays as input for external functions which are unit-naive.
This can be done safely by first converting to the target unit, then
dequantifying the dataset or array:

In [ ]:
ds["CH4"].pint.to("Mt CH4 / year").pr.dequantify()

Note that the units are then stored in the DataArray's `attrs`, and can be
restored using the `da.pr.quantify()` function.

## Aggregation and infilling

xarray provides robust functions for aggregation (`sum`) and filling of
missing information (`fillna`).
PRIMAP2 adds functions which fill or skip missing information based on if the
information is missing at all points along certain axes, for example for
a whole time series.
This makes it possible to, for example, evaluate the sum of sub-categories
while ignoring only those categories which are missing completely.

In [ ]:
import numpy as np
import xarray as xr
import pandas as pd

time = pd.date_range("2000-01-01", "2003-01-01", freq="AS")
area_iso3 = np.array(["COL", "ARG", "MEX"])
coords = [("area (ISO3)", area_iso3), ("time", time)]
da = xr.DataArray(
    data=[
        [1, 2, 3, 4],
        [np.nan, np.nan, np.nan, np.nan],
        [1, 2, 3, np.nan],
    ],
    coords=coords,
)

da

In [ ]:
da.pr.sum_skip_all_na(dim="area (ISO3)", skipna_evaluation_dims="time")

In [ ]:
# compare this to the result of the standard xarray sum:

da.sum(dim="area (ISO3)")

The same functionality is available for filling in missing information:

In [ ]:
da.pr.fill_all_na("time", value=10)

## Handling of gas baskets

TODO: gas_basket_contents_sum, fill_na_gas_basket_from_contents